In [9]:
import pandas as pd
import datatable as dt

# Variables that contains the file location
from files import *
from functions import *

In [10]:
# if we modify the file we need to reload it with this
import importlib
import functions  #import the module here, so that it can be reloaded.

importlib.reload(functions)

<module 'functions' from '/DATA/myprograms_Python/MMSR/project/functions.py'>

In [11]:
DATA_TYPE = np.float16

# Data
Load the Data

In [12]:
genres = dt.fread(file_genres_2).to_pandas()
genres.set_index('id', inplace=True)

In [18]:
# Our special child
blf_logfluc = dt.fread(file_blf_logfluc)
blf_logfluc[dt.float64] = dt.float32
new_cols = ['id']
new_cols.extend(list(blf_logfluc.names[2:]))
new_cols = tuple(new_cols)
del blf_logfluc[:, -1]
blf_logfluc.names = new_cols
blf_logfluc = blf_logfluc.to_pandas()
blf_logfluc.set_index('id', inplace=True)

In [6]:
files = [
    # Lyrics
    file_tfidf_2,
    file_word2vec_2,
    file_bert_2,
    # Audio
    # file_essentia, # zeros
    file_blf_correlation,
    file_blf_deltaspectral,
    file_blf_spectral,
    # file_blf_spectralcontrast, # worst performance, skipped to free cpu time
    file_blf_vardeltaspectral,
    # file_blf_logfluc, # zeros
    blf_logfluc,
    file_mfcc_bow,
    file_mfcc_stats,
    # Video
    file_incp,
    file_resnet,
    file_vgg19,
]

In [7]:
import csv

# Load the data, using pandas csv reader instead of datatables to get access to float16
data = []
for file in tqdm(files, desc="Loading data"):
    if type(file) == str:
        with open(file, "r") as f:
            column_names = next(csv.reader(f, delimiter="\t"))
        dtypes = {
            x: DATA_TYPE
            for x in column_names
            if x != "id"
        }
        file = pd.read_csv(file, dtype=dtypes, index_col=0, delimiter="\t")
    data.append(file)

Loading data: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [03:07<00:00, 14.45s/it]


In [8]:
# Extract readable names
names = []
for file in files:
    if type(file) == str:
        names.append(file.replace("./../task2/id_", "").replace(".tsv", ""))
    else:
        names.append("blf_logfluc")

In [9]:
for name, file in zip(names, data):
    print(name, file.shape)

lyrics_tf-idf_mmsr (68641, 1000)
lyrics_word2vec_mmsr (68641, 300)
lyrics_bert_mmsr (68641, 768)
blf_correlation_mmsr (68641, 1326)
blf_deltaspectral_mmsr (68641, 1372)
blf_spectral_mmsr (68641, 980)
blf_vardeltaspectral_mmsr (68641, 1344)
blf_logfluc (68641, 3626)
mfcc_bow_mmsr (68641, 500)
mfcc_stats_mmsr (68641, 104)
incp_mmsr (68641, 4096)
resnet_mmsr (68641, 4096)
vgg19_mmsr (68641, 8192)


# Late Fusion

In [10]:
def compute_in_batches_topIds(results: np.array, idx_values: np.array, top: int = 100, batches: int = 1):
    splits_b = np.array_split(results, batches, axis=0)
    return np.concatenate([idx_values[np.argsort(b * -1, axis=1)][:, :top] for b in tqdm(splits_b)], axis=0)

In [11]:
def compute_in_batches(arr_a: np.array, arr_b: np.array, simfunction, batches: int = 1):
    # arr_a: full Data array
    # arr_b: full Data array arr_a or just some songs
    # batches: split arr_b into chunks for multiprocess and less RAM usage
    splits_b = np.array_split(arr_b, batches, axis=0)
    r = []
    for b in tqdm(splits_b):
        r.append(simfunction(arr_a, b).astype(DATA_TYPE))
    return np.concatenate(r, axis=1, dtype=DATA_TYPE)

# Find optimal feature similarity weights

In [12]:
subset = np.random.choice(len(data[0]), 1024 * 4, replace=False)
subset.shape

(4096,)

In [13]:
subset_ids = data[0].index.values[subset]

similarities = []
functions = [get_cosine_similarity]

# for every data
i = 0
for d in tqdm(data, desc="Processing"):
    file = d.to_numpy()[subset, :].astype(np.float32)
    for f in functions:
        result = np.nan_to_num(f(file, file), copy=False)
        similarities.append(result)
    i += 1

print(f"{len(similarities)} similarity matrices")

Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:04<00:00,  3.15it/s]

13 similarity matrices


In [14]:
def evaluate(similarities, weights):
    if len(similarities) == 1:
        accumulated_similarity = similarities[0]
    else:
        accumulated_similarity = np.zeros((len(subset), len(subset)), dtype=DATA_TYPE)
        for sim, weight in zip(similarities, weights):
            accumulated_similarity += (sim * weight)

    top_ids = pd.DataFrame(compute_in_batches_topIds(accumulated_similarity, subset_ids, 100, 10), subset_ids)

    return {
        # "MAP@10": meanAveragePrecision(top_ids, 10, genres),
        # "MAP@100": meanAveragePrecision(top_ids, 100, genres),
        # "MRR@10": meanReciprocalRank(top_ids, 10, genres),
        # "MRR@100": meanReciprocalRank(top_ids, 100, genres),
        "NDCG@10": ndcgMean(top_ids, 10, genres)[1],
        "NDCG@100": ndcgMean(top_ids, 100, genres)[1],
    }

In [15]:
print("Baseline", evaluate(similarities, np.zeros((len(similarities),))))

  0%|                                                                                                                                                                                                              | 0/4096 [00:00<?, ?it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(

/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: Run

/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: Run

 39%|███████████████████████████████████████████████████████████████████████████▉                                                                                                                     | 1611/4096 [00:01<00:01, 1598.22it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(

/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: Run

...

/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍          | 3872/4096 [00:02<00:00, 1621.59it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(

/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
 11%|█████████████████████▍                                                                                                                                                                             | 450/4096 [00:00<00:05, 625.68it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
 13%|████████████████████████▋                                                                                                                                                                          | 518/4096 [00:00<00:05, 642.26it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: Ru

 46%|██████████████████████████████████████████████████████████████████████████████████████████▏                                                                                                       | 1904/4096 [00:02<00:03, 635.71it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
 48%|█████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                    | 1971/4096 [00:03<00:03, 642.52it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: Ru

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                       | 3267/4096 [00:05<00:01, 635.53it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
 81%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                    | 3331/4096 [00:05<00:01, 634.78it/s]/DATA/myprograms_Python/MMSR/project/functions.py:109: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                 | 3398/4096 [00

Baseline {'NDCG@10': nan, 'NDCG@100': nan}


In [16]:
print("Fair", evaluate(similarities, np.ones((len(similarities),))))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4096/4096 [00:06<00:00, 617.73it/s]

Fair {'NDCG@10': 0.871684092394027, 'NDCG@100': 0.8086574970605949}


In [17]:
weights = np.zeros((len(similarities),))
for i, sim in enumerate(similarities):
    result = np.nan_to_num(np.asarray(list(evaluate([sim], [1]).values())), copy=False)
    weights[i] = np.mean(result, where=result>0)
print(weights)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.65it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4096/4096 [00:06<00:00, 604.04it/s]

[0.84035388 0.8246092  0.8557342  0.82346927 0.82595463 0.82725968
 0.82571856 0.82847185 0.82576923 0.82900186 0.82358706 0.82458641
 0.8220057 ]


In [18]:
print("Performance", evaluate(similarities, weights))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4096/4096 [00:06<00:00, 630.22it/s]

Performance {'NDCG@10': 0.8714683776560417, 'NDCG@100': 0.8092064373653168}


In [19]:
norm_weights = (weights - weights.min()) / (weights.max() - weights.min())
print(norm_weights)
print("Normalized Performance", evaluate(similarities, norm_weights))

[0.54399645 0.07718987 1.         0.04339271 0.11707986 0.15577277
 0.11008061 0.19171158 0.11158316 0.20742574 0.04688486 0.07651424
 0.        ]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4096/4096 [00:06<00:00, 635.27it/s]

Normalized Performance {'NDCG@10': 0.8847473691774442, 'NDCG@100': 0.8292093374043561}


# Final Processing
Normalized Performance seems to be the best approach

In [19]:
final_ids = blf_logfluc.index.values
final_accumulated_similarity = np.zeros((len(final_ids), len(final_ids)), dtype=DATA_TYPE)
functions = [get_cosine_similarity]

In [20]:
# for every data
i = 0
for d in tqdm(data, desc="Processing"):
    file = d.to_numpy().astype(np.float32)
    for f in functions:
        result = compute_in_batches(file, file, f, 100)
        if norm_weights[i] > 0:
            result = np.nan_to_num(result, copy=False)
            result = np.multiply(result, norm_weights[i], out=result)
            np.add(final_accumulated_similarity, result, out=final_accumulated_similarity)
        print("Sanity Check", final_accumulated_similarity[:1024, :1024].mean())
        i += 1

 32%|███████████████████████████████████████████████████████████████▎                                                                                                                                      | 32/100 [01:12<03:04,  2.72s/it]


 66%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                   | 66/100 [02:48<01:24,  2.49s/it]


Processing:   8%|██████████████▏                                                                                                                                                                          | 1/13 [09:21<1:52:14, 561.25s/it]

Sanity Check 0.03055



 33%|█████████████████████████████████████████████████████████████████▎                                                                                                                                    | 33/100 [00:33<01:14,  1.12s/it]


...

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                 | 67/100 [01:44<01:06,  2.01s/it]


Processing:  77%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                          | 10/13 [1:53:18<34:44, 694.96s/it]

Sanity Check 1.327



Processing:  77%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                          | 10/13 [1:53:27<34:02, 680.73s/it]


KeyboardInterrupt: 

In [21]:
np.save("final_similarity.p", final_accumulated_similarity)

In [26]:
final_accumulated_similarity = np.load("final_similarity.p.npy")

In [27]:
final_accumulated_similarity.mean()

1.324

In [28]:
top_ids = pd.DataFrame(compute_in_batches_topIds(final_accumulated_similarity, final_ids, 100, 100), final_ids)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:13<00:00,  3.73s/it]


In [29]:
dt.Frame(pd.DataFrame(top_ids, index=final_ids).reset_index()).to_csv('./top_ids_late_fusion.csv')

In [30]:
results = {
    #"MAP@10": meanAveragePrecision(top_ids, 10, genres),
    #"MAP@100": meanAveragePrecision(top_ids, 100, genres),
    #"MRR@10": meanReciprocalRank(top_ids, 10, genres),
    #"MRR@100": meanReciprocalRank(top_ids, 100, genres),
    "NDCG@10": ndcgMean(top_ids, 10, genres)[1],
    "NDCG@100": ndcgMean(top_ids, 100, genres)[1],
}

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68641/68641 [01:51<00:00, 614.29it/s]


In [31]:
results

{'NDCG@10': 0.8637549146518158, 'NDCG@100': 0.7889160890493015}